In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import pandas as pd


import sys
import xlwt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import datetime,pickle,os,glob



def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y




def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList():
    with open('pickles/stationList.pickle', 'rb') as handle:
        stationList = pickle.load(handle)
    os.chdir('excelFiles/LSTM')
    replaceDict = ['.xls','LSTMresult']
    for direct in glob.glob("*.xls"):
        fileName = direct                                               
        for w in replaceDict:
            fileName = fileName.replace(w,'')
        
        if fileName in stationList:
            stationList.remove(fileName)
            print(fileName)
    os.chdir('../..')
    return stationList

def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(station,windosSize):

    with open('pickles/'+station+'2017trainRaw.pickle', 'rb') as handle:
        trainRawData = pickle.load(handle)
    with open('pickles/'+station+'2017testRaw.pickle', 'rb') as handle:
        testRawData = pickle.load(handle)
        
    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainLR(sc, X_train, y_train, X_test, y_test):
    reg = LinearRegression().fit(X_train, y_train)
    predicted_LR = sc.inverse_transform(reg.predict(X_test))
    ori_y_test = sc.inverse_transform(y_test)
    mse = mean_squared_error(predicted_LR, ori_y_test)
    print(mse)
    return mse

In [6]:


stationList = getStationList()
col=1
for station in stationList:
    print("training : " +station)
    mseList = []
    for windowSize in range(7,17):
        sc, X_train, y_train, X_test, y_test = fetchData(station,windowSize)
        mse = trainLR(sc, X_train, y_train, X_test, y_test)
        mseList.append(mse)
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,10,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    book.save("excelFiles/LR/LRresult"+station+".xls")


training : 二林
24.67836090372839
38.52563500414971
53.60278095440399
65.04802261015914
74.67271637440574
82.87533296638968
89.86799687347978
96.05761448877679
101.57382337312555
106.44696434055479
training : 南投
15.574554692608368
30.322600204810062
42.59784225867757
53.27123872588267
62.57729642484003
70.63109405137878
77.69644254540967
83.99306223196888
89.71852156267498
94.89157358010698
training : 埔里
12.842479261260094
27.6522861861239
40.48942642639641
52.104208271193485
62.57568655764194
71.94320902067612
80.33106880493297
87.85346999573662
94.56001312531559
100.55934103691872
training : 大里
29.007482210805627
42.16656672967937
56.823947289600596
66.47648558495074
74.52930238116744
81.2961462331824
87.10789882293969
92.19525013748765
96.69302628109101
100.70894371032313
training : 彰化
43.58995678705991
64.91938837088375
89.81055958220759
107.08088752443663
121.61185359397084
133.60407191485487
143.4388237337533
151.5777095782362
158.57636242674803
164.80249215687428
training : 忠明
23.

79.4662106176407
83.78327337178105
training : 苗栗
23.059215113134552
32.08221321227114
39.93372821822512
46.6460962345833
52.28596562544701
57.103182018499666
61.24846095855535
64.86139133312189
68.0117806948356
70.82882265914597
training : 頭份
13.744828751087152
25.46635844686419
34.623610255245715
42.33897650480965
48.75919480406576
54.18728603326031
58.83022472685313
62.81541540860983
66.26436497422027
69.28649361220538
training : 臺東
4.758398107248661
7.643069444206784
9.159927733673475
10.272818949046567
11.228191319393819
12.087408372247237
12.869870208150626
13.597556480979328
14.266298351174962
14.893741319366328
training : 花蓮
9.401632895884191
14.999913384873025
18.35876179395892
20.947653018641454
23.05289523016349
24.852212777632804
26.37133909491508
27.669598061704264
28.804468701733526
29.807653556325327
training : 關山
3.6906897691738227
6.190151538996748
7.824265132607603
9.127867420746579
10.187838256405229
11.064120357278341
11.812940462508651
12.462412598606033
13.03225751

NameError: name 'db' is not defined

In [12]:
k = [1,2,3,4,5,6,7,8,9]
k[-6:]

[4, 5, 6, 7, 8, 9]

In [10]:
k = [1,2,3,4,5]
k[2:]



[3, 4, 5]

In [ ]:
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])